# Data augmentations to reduce the domain shift 
A naive solution to improve the generalization capability of the segmentation network trained on the synthetic domain consists in the usage of data augmentations during training. Through them, we i) virtually expand the dataset size and ii) modify the visual appearance of source (synthetic) images in order to make them more similar to the target (real) ones. 
Specifically, we repeat the previous experiment, introducing data augmentations at training time (e.g. horizontal flip, Gaussian Blur, Multiply, ecc.). The decision of what kind of algorithm is left to the student. Set the probability to perform augmentation to 0.5.




In [1]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 791.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=15caf3df2f3581e8c11eb7cc4f5c3a40a6c9f4025492c53bd2308efcee751ab2
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=a183fed79cd00524a9e24941eef7f049c0f88991990695b3cf1bff9cd9dafb6f
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [2]:
# WANDB
!pip install -q wandb

In [3]:
# If you run the model for the first time remove all the previus checkpoints
! rm -r checkpoints/

rm: cannot remove 'checkpoints/': No such file or directory


# 0 - Import libraries

In [4]:
import wandb

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

import os
import zipfile
import numpy as np
import time
from PIL import Image

from fvcore.nn import FlopCountAnalysis, flop_count_table

import warnings
warnings.filterwarnings(action='ignore')

# 1 - Start WanDB

In [5]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 2 - Model Pipeline

In [6]:
def model_pipeline(hyperparameters=None):

    # tell wandb to get started
    with wandb.init(project="MLDL-step3a", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, val_loader, criterion, optimizer, start_epoch = make(config)
        
        # and use them to train the model
        train(model, train_loader, criterion, optimizer, config, start_epoch)

        # and test its final performance
        val(model, val_loader)

    return model

# 3 - Datasets

In [7]:
def make(config):
    # Make the data
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size,train=True)
    test_loader = make_loader(test, batch_size=config.batch_size,train=False)

    # Make the model (BiSeNet with ResNet-18 backbone)
    model = build_model(model_type='BiSeNet').cuda()

    # Make the loss and optimizer
    optimizer = optim.SGD(model.parameters(), 
                          lr=config.learning_rate, 
                          momentum=config.momentum, 
                          weight_decay=config.weight_decay)
    
    criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
    
    # Load the last checkpoint
    start_epoch = load_checkpoint(config, model, optimizer)
    
    return model, train_loader, test_loader, criterion, optimizer, start_epoch

In [8]:
import torchvision.transforms as transforms

# Define augmentation transformations for training
augmentation_transforms_train = [
    transforms.RandomHorizontalFlip(p=0.5),  # Horizontal flip with probability 0.5
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jittering
    transforms.RandomResizedCrop(size=(512, 1024), scale=(0.5, 1.0), ratio=(0.75, 1.333))  # Random crop and resize
]

# Define base transformations for validation/test
base_transforms_city = [
    transforms.Resize((512, 1024)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]

base_transforms_gta = [
    transforms.Resize((720, 1280)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]

# Define transforms for preprocessing
image_transform_cityescapes = {
    'train': transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.RandomApply(augmentation_transforms_train, p=0.5),  # Apply augmentation with probability 0.5
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose(base_transforms_city)
}

image_transform_gta = {
    'train': transforms.Compose(base_transforms_gta),  # Use base transforms for training GTA5
    'val': transforms.Compose(base_transforms_gta)
}

label_transform_cityescapes = transforms.Compose([
    transforms.Resize((512, 1024))
])

label_transform_gta = transforms.Compose([
    transforms.Resize((720, 1280))
])

# GTA5 for train and CityScapes for test
cityscapes_dir = '/kaggle/input/cityscapes/Cityscapes/Cityspaces'
gta_dir = '/kaggle/input/gta5-dataset-with-masks'

def get_data(train=True):
    if train:
        # train dataset
        dataset = GTA5(root_dir=gta_dir, image_transform=image_transform_gta['train'], label_transform=label_transform_gta)
    else:
        # test dataset
        dataset = CityScapes(root_dir=cityscapes_dir, split='val', image_transform=image_transform_cityescapes['val'], label_transform=label_transform_cityescapes)
    
    return dataset

def make_loader(dataset, batch_size = 8, train=True):
    if train == True:
        # train dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
    else:
        # test dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False,drop_last=True)
    
    return dataloader



In [9]:
# TO IMPLEMENT
def build_model(model_type):
    if model_type == 'BiSeNet':
        return BiSeNet(num_classes=19, context_path="resnet18")
    elif model_type == 'DeepLabV2':
        pretrain_model_path = '/kaggle/input/deeplab_v2_model/pytorch/model_weight/1/deeplab_resnet_pretrained_imagenet.pth'
        return get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path=pretrain_model_path)

## 3.1 - CityScapes

In [10]:
class CityScapes(Dataset):
    def __init__(self, root_dir, split='train', image_transform=None, label_transform=None):
        super(CityScapes, self).__init__()
        """
        Args:
            root_dir (string): Directory with all the images and annotations.
            split (string): 'train' or 'val'.
            transform (callable, optional): Optional transform to be applied on a sample.
        """

        self.root_dir = root_dir
        self.split = split
        self.image_transform = image_transform
        self.label_transform = label_transform

        # Get the image and label directories
        self.image_dir = os.path.join(root_dir, 'images', split)
        self.label_dir = os.path.join(root_dir, 'gtFine', split)

        # Get a list of all image files
        self.image_files = []
        for city_dir in os.listdir(self.image_dir):
            city_image_dir = os.path.join(self.image_dir, city_dir)
            self.image_files.extend([os.path.join(city_image_dir, f) for f in os.listdir(city_image_dir) if f.endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # Get the corresponding label image path
        label_name = img_name.replace('images', 'gtFine').replace('_leftImg8bit', '_gtFine_labelTrainIds')

        # Load image and label
        image = Image.open(img_name).convert('RGB')
        label = Image.open(label_name).convert('L')

        if self.image_transform:
            image = self.image_transform(image)
        if self.label_transform:
            label = self.label_transform(label)

        label = torch.Tensor(np.array(label))

        return image, label

## GTA5

In [11]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import torchvision.transforms as T

class GTA5(Dataset):
    def __init__(self, root_dir, image_transform=None, label_transform=None):
        super(GTA5, self).__init__()
        """
        Args:
            root_dir (string): Directory with all the images and annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """

        self.root_dir = root_dir
        self.image_transform = image_transform
        self.label_transform = label_transform

        # Get the image and label directories
        self.image_dir = os.path.join(root_dir, 'images')
        self.label_dir = os.path.join(root_dir, 'masks')

        # Get a list of all image files
        self.image_files = []
        # Get a list of all files in the images directory
        for file_name in os.listdir(self.image_dir):
            file_path = os.path.join(self.image_dir, file_name)
            if os.path.isfile(file_path):
                self.image_files.append(file_name)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name, label_name = self.image_files[idx], self.image_files[idx]
        
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, label_name)

        # Load image and label
        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('L')
        
        if self.image_transform:
            image = self.image_transform(image)
        if self.label_transform:
            label = self.label_transform(label)

        label = torch.Tensor(np.array(label)) 

        return image, label

# 4 - BiSeNet

In [12]:
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        x = self.conv1(input)
        return self.relu(self.bn(x))


class Spatial_path(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = ConvBlock(in_channels=3, out_channels=64)
        self.convblock2 = ConvBlock(in_channels=64, out_channels=128)
        self.convblock3 = ConvBlock(in_channels=128, out_channels=256)

    def forward(self, input):
        x = self.convblock1(input)
        x = self.convblock2(x)
        x = self.convblock3(x)
        return x


class AttentionRefinementModule(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.sigmoid = nn.Sigmoid()
        self.in_channels = in_channels
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input):
        # global average pooling
        x = self.avgpool(input)
        assert self.in_channels == x.size(1), 'in_channels and out_channels should all be {}'.format(x.size(1))
        x = self.conv(x)
        x = self.sigmoid(self.bn(x))
        # x = self.sigmoid(x)
        # channels of input and x should be same
        x = torch.mul(input, x)
        return x


class FeatureFusionModule(torch.nn.Module):
    def __init__(self, num_classes, in_channels):
        super().__init__()
        # self.in_channels = input_1.channels + input_2.channels
        # resnet101 3328 = 256(from spatial path) + 1024(from context path) + 2048(from context path)
        # resnet18  1024 = 256(from spatial path) + 256(from context path) + 512(from context path)
        self.in_channels = in_channels

        self.convblock = ConvBlock(in_channels=self.in_channels, out_channels=num_classes, stride=1)
        self.conv1 = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input_1, input_2):
        x = torch.cat((input_1, input_2), dim=1)
        assert self.in_channels == x.size(1), 'in_channels of ConvBlock should be {}'.format(x.size(1))
        feature = self.convblock(x)
        x = self.avgpool(feature)

        x = self.relu(self.conv1(x))
        x = self.sigmoid(self.conv2(x))
        x = torch.mul(feature, x)
        x = torch.add(x, feature)
        return x


class BiSeNet(torch.nn.Module):
    def __init__(self, num_classes, context_path):
        super().__init__()
        # build spatial path
        self.saptial_path = Spatial_path()

        # build context path
        self.context_path = build_contextpath(name=context_path)

        # build attention refinement module  for resnet 101
        if context_path == 'resnet101':
            self.attention_refinement_module1 = AttentionRefinementModule(1024, 1024)
            self.attention_refinement_module2 = AttentionRefinementModule(2048, 2048)
            # supervision block
            self.supervision1 = nn.Conv2d(in_channels=1024, out_channels=num_classes, kernel_size=1)
            self.supervision2 = nn.Conv2d(in_channels=2048, out_channels=num_classes, kernel_size=1)
            # build feature fusion module
            self.feature_fusion_module = FeatureFusionModule(num_classes, 3328)

        elif context_path == 'resnet18':
            # build attention refinement module  for resnet 18
            self.attention_refinement_module1 = AttentionRefinementModule(256, 256)
            self.attention_refinement_module2 = AttentionRefinementModule(512, 512)
            # supervision block
            self.supervision1 = nn.Conv2d(in_channels=256, out_channels=num_classes, kernel_size=1)
            self.supervision2 = nn.Conv2d(in_channels=512, out_channels=num_classes, kernel_size=1)
            # build feature fusion module
            self.feature_fusion_module = FeatureFusionModule(num_classes, 1024)
        else:
            print('Error: unspport context_path network \n')

        # build final convolution
        self.conv = nn.Conv2d(in_channels=num_classes, out_channels=num_classes, kernel_size=1)

        self.init_weight()

        self.mul_lr = []
        self.mul_lr.append(self.saptial_path)
        self.mul_lr.append(self.attention_refinement_module1)
        self.mul_lr.append(self.attention_refinement_module2)
        self.mul_lr.append(self.supervision1)
        self.mul_lr.append(self.supervision2)
        self.mul_lr.append(self.feature_fusion_module)
        self.mul_lr.append(self.conv)

    def init_weight(self):
        for name, m in self.named_modules():
            if 'context_path' not in name:
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                elif isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-5
                    m.momentum = 0.1
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

    def forward(self, input):
        # output of spatial path
        sx = self.saptial_path(input)

        # output of context path
        cx1, cx2, tail = self.context_path(input)
        cx1 = self.attention_refinement_module1(cx1)
        cx2 = self.attention_refinement_module2(cx2)
        cx2 = torch.mul(cx2, tail)
        # upsampling
        cx1 = torch.nn.functional.interpolate(cx1, size=sx.size()[-2:], mode='bilinear')
        cx2 = torch.nn.functional.interpolate(cx2, size=sx.size()[-2:], mode='bilinear')
        cx = torch.cat((cx1, cx2), dim=1)

        if self.training == True:
            cx1_sup = self.supervision1(cx1)
            cx2_sup = self.supervision2(cx2)
            cx1_sup = torch.nn.functional.interpolate(cx1_sup, size=input.size()[-2:], mode='bilinear')
            cx2_sup = torch.nn.functional.interpolate(cx2_sup, size=input.size()[-2:], mode='bilinear')

        # output of feature fusion module
        result = self.feature_fusion_module(sx, cx)

        # upsampling
        result = torch.nn.functional.interpolate(result, scale_factor=8, mode='bilinear')
        result = self.conv(result)

        if self.training == True:
            return result, cx1_sup, cx2_sup

        return result

In [13]:
class resnet18(torch.nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.features = models.resnet18(pretrained=pretrained)
        self.conv1 = self.features.conv1
        self.bn1 = self.features.bn1
        self.relu = self.features.relu
        self.maxpool1 = self.features.maxpool
        self.layer1 = self.features.layer1
        self.layer2 = self.features.layer2
        self.layer3 = self.features.layer3
        self.layer4 = self.features.layer4

    def forward(self, input):
        x = self.conv1(input)
        x = self.relu(self.bn1(x))
        x = self.maxpool1(x)
        feature1 = self.layer1(x)  # 1 / 4
        feature2 = self.layer2(feature1)  # 1 / 8
        feature3 = self.layer3(feature2)  # 1 / 16
        feature4 = self.layer4(feature3)  # 1 / 32
        # global average pooling to build tail
        tail = torch.mean(feature4, 3, keepdim=True)
        tail = torch.mean(tail, 2, keepdim=True)
        return feature3, feature4, tail


class resnet101(torch.nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.features = models.resnet101(pretrained=pretrained)
        self.conv1 = self.features.conv1
        self.bn1 = self.features.bn1
        self.relu = self.features.relu
        self.maxpool1 = self.features.maxpool
        self.layer1 = self.features.layer1
        self.layer2 = self.features.layer2
        self.layer3 = self.features.layer3
        self.layer4 = self.features.layer4

    def forward(self, input):
        x = self.conv1(input)
        x = self.relu(self.bn1(x))
        x = self.maxpool1(x)
        feature1 = self.layer1(x)  # 1 / 4
        feature2 = self.layer2(feature1)  # 1 / 8
        feature3 = self.layer3(feature2)  # 1 / 16
        feature4 = self.layer4(feature3)  # 1 / 32
        # global average pooling to build tail
        tail = torch.mean(feature4, 3, keepdim=True)
        tail = torch.mean(tail, 2, keepdim=True)
        return feature3, feature4, tail


def build_contextpath(name):
    model = {
        'resnet18': resnet18(pretrained=True),
        'resnet101': resnet101(pretrained=True)
    }
    return model[name]

# 5 - Training

In [14]:
def train(model, dataloader, criterion, optimizer, config, start_epoch):
    
    for epoch in range(start_epoch, config.epochs):
        running_loss = 0.0
        total_mIOU = 0
        total_images = 0
        
        for _, (inputs, targets) in enumerate(dataloader):

            inputs, targets = inputs.cuda(), id_processing(targets).cuda()

            outputs = model(inputs)

            loss = criterion(outputs[0], targets)

            # Backprpagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs[0].max(1)

            running_mIOU = mean_iou(outputs[0].size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)

            poly_lr_scheduler(optimizer, config.learning_rate, iter=epoch, max_iter=config.epochs)
                
        train_loss = running_loss / len(dataloader)
        mIOU = total_mIOU/total_images
        
        # Save the train metrics by using wandb
        train_log(train_loss, mIOU, epoch)
        
        # Save checkpoint (overwrite)
        save_checkpoint(config, model, optimizer, train_loss, mIOU, epoch)

In [15]:
def id_processing(targets):
    targets = targets.cuda()
    
    # Define valid indices
    valid_indices = torch.tensor(list(range(19)) + [255]).to(targets.device)

    # Replace all IDs not in valid_indices with 255
    processed_targets = torch.where(torch.isin(targets, valid_indices), targets, torch.tensor(255, device=targets.device))
    
    # Release GPU memory IMPORTANT
    del targets
    torch.cuda.empty_cache()

    return processed_targets.long()

In [16]:
def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter=1,
                      max_iter=300, power=0.9):
    """Polynomial decay of learning rate
            :param init_lr is base learning rate
            :param iter is a current iteration
            :param lr_decay_iter how frequently decay occurs, default is 1
            :param max_iter is number of maximum iterations
            :param power is a polymomial power

    """
    # if iter % lr_decay_iter or iter > max_iter:
    # 	return optimizer

    lr = init_lr*(1 - iter/max_iter)**power
    optimizer.param_groups[0]['lr'] = lr
    return lr

def mean_iou(num_classes, pred, target):
    mIOU = 0
    for i in range(len(pred)):
        hist = fast_hist(target[i].cpu().numpy(),pred[i].cpu().numpy(), num_classes)
        IOU = per_class_iou(hist)
        mIOU = mIOU + sum(IOU)/num_classes
    return mIOU

def fast_hist(a, b, n):
    """
    a and b are predict and mask respectively
    n is the number of classes
    """
    k = (a >= 0) & (a < n) #assign True if the value is in the range between 0 and 18 (class labels)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape((n, n))

def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

In [17]:
def train_log(loss, mIOU, epoch):
    #wandb.log({"epoch": epoch, "loss": loss, "mIOU":mIOU})
    wandb.log({"loss": loss, "mIOU":mIOU}, step= epoch)
    print(f"----------------------------------")
    print(f"Loss after {epoch} epochs: {loss:.3f}")
    print(f"mIOU after {epoch} epochs: {mIOU:.3f}%")

In [18]:
def save_checkpoint(config, model, optimizer, train_loss, mIOU, epoch):
    checkpoint_path = os.path.join(config.checkpoint_dir, "checkpoint.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': train_loss,
        'mIOU': mIOU
    }, checkpoint_path)
    print(f"Checkpoint saved in {checkpoint_path} | Epoch: {epoch}")
    
    
def load_checkpoint(config, model, optimizer):
    if os.path.exists(config.checkpoint_dir):
        checkpoint = torch.load(config.checkpoint_dir + "/checkpoint.pth")
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint found. Resuming from epoch {start_epoch}.")
        return start_epoch
    else:
        os.mkdir(config.checkpoint_dir) # divide the directory wrt the model (eg. checkpoints/DeepLabV2, checkpoints/BiSeNet)
        print("No checkpoint found. Starting from scratch.")
        return 0

# 6 - Validation

In [19]:
# Validation method
def val(model, dataloader):
    model.eval()
    total_mIOU = 0
    total_images = 0
    latency_list = []
    FPS_list = []
    
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), id_processing(targets).cuda()
            
            start = time.time() # Record start time
            outputs = model(inputs)
            end = time.time() # Record end time

            # Calculate latency for this iteration
            latency_i = end - start
            latency_list.append(latency_i)

            # Calculate FPS for this iteration
            FPS_i = 1 / latency_i
            FPS_list.append(FPS_i)

            _, predicted = outputs.max(1)
            
            running_mIOU = mean_iou(outputs.size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)
        
    mIOU = total_mIOU/total_images
    latency = np.sum(latency_list) / len(latency_list)
    test_FPS = np.sum(FPS_list) / len(FPS_list)
    
    # compute flops and #param
    image, _ = next(iter(dataloader))
    height, width = image.shape[2], image.shape[3]
    zero_image = torch.zeros((1, 3, height, width))
    flops = FlopCountAnalysis(model, zero_image.cuda())
    print(flop_count_table(flops))

    print(f'\n\nmIoU: {(mIOU*100):.3f}%, Latency: {latency:.3f}, FPS: {test_FPS:.3f}')

    wandb.log({"mIOU":mIOU,"latency":latency,"FPS":test_FPS})

# 7 - Hyperparameter Sweeps using WanDB

In [32]:
sweep_config= {
    'name': 'DomainAdaptation-sweep',
    'metric': {'name': 'loss', 'goal': 'minimize'}, # the goal is maximize the accuracy
    'method': 'random', # test all possible combinations of the hyperparameters
    'parameters': {
        'epochs': {'values': [5]},        
        'learning_rate': {'values': [0.1, 0.001, 0.0001]}, # 2 parameters to optimize during the sweep
        'batch_size': {'values': [2, 4, 8]},
        'momentum': {'values': [0.9]},
        'weight_decay': {'values': [5e-4]}
    }
}

In [27]:
sweep_id = wandb.sweep(sweep=sweep_config, project="MLDL-step3a")

Create sweep with ID: s426arij
Sweep URL: https://wandb.ai/polito-tmazzarini/mldl_step2b/sweeps/s426arij


In [28]:
if torch.cuda.is_available():
    print("Start hyperparameter sweeps\n")
    wandb.agent(sweep_id, function=model_pipeline, count=10)
else:
    print("CUDA is Not available")

wandb: Agent Starting Run: x6jq0rxd with config:
wandb: 	batch_size: 2
wandb: 	epoch: 5
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	weight_decay: 0.0005
wandb: ERROR Run x6jq0rxd errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR TypeError: model_pipeline() missing 1 required positional argument: 'hyperparameters'
wandb: ERROR 


# 8 - Build final model

In [20]:
# best configuration (TO CONFIGURE)
config = dict(
    epochs=50,
    batch_size=2,
    learning_rate=0.001,
    momentum=0.9,
    weight_decay=5e-4,
    architecture="BiSeNet",
    checkpoint_dir="/kaggle/working/checkpoints")

In [21]:
if torch.cuda.is_available():
    print("Building the model with the best configuration")
    # Build, train and analyze the model with the pipeline
    model = model_pipeline(config)
else:
    print("CUDA is Not available")

wandb: Currently logged in as: tommasomazzarini2001 (polito-tmazzarini). Use `wandb login --relogin` to force relogin


Building the model with the best configuration


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s] 
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 158MB/s]  


No checkpoint found. Starting from scratch.


Traceback (most recent call last):
  File "/tmp/ipykernel_34/2884409676.py", line 12, in model_pipeline
    train(model, train_loader, criterion, optimizer, config, start_epoch)
  File "/tmp/ipykernel_34/1525349719.py", line 25, in train
    running_mIOU = mean_iou(outputs[0].size()[1], predicted, targets)
  File "/tmp/ipykernel_34/2562121041.py", line 4, in mean_iou
    hist = fast_hist(target[i].cpu().numpy(),pred[i].cpu().numpy(), num_classes)
  File "/tmp/ipykernel_34/2562121041.py", line 15, in fast_hist
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape((n, n))
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
wandb.finish()

# 9 - Save model weights

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

# 10 - Model Application

In [ ]:
# Lad the model 
model =  pass # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

In [1]:
# pick une image
# use model and predict the mask
# map each class to a colored value
# plot the segmented image